# Analisis Jumlah dan Hubungan Streamer pada Twitch dengan Algoritma Degree Centrality dan Strongly Connected Components

In [ ]:
pip install neo4j

In [ ]:
pip install neo4j-driver

In [ ]:
pip install pandas


In [ ]:
from neo4j import GraphDatabase, basic_auth

In [ ]:
#connect project/dataset from Neo4j

driver = GraphDatabase.driver(
  "bolt://54.210.12.215:7687",
  auth=basic_auth("neo4j", "paths-wrist-mover"))

In [ ]:
from pandas import DataFrame

#Degree Centrality

dengan menggunakan algorithms ini bertujuan untuk menentukan perhitungan total chatter/moderator yang masuk pada streamer

## Chatter (w/ User)

In [ ]:
with driver.session() as session:
    result = session.run("CALL gds.graph.create('myGraph31','User', {CHATTER: { orientation: 'REVERSE'}})")
    for r in result:
        print(r)
      

<Record nodeProjection={'User': {'properties': {}, 'label': 'User'}} relationshipProjection={'CHATTER': {'orientation': 'REVERSE', 'aggregation': 'DEFAULT', 'type': 'CHATTER', 'properties': {}}} graphName='myGraph31' nodeCount=4678779 relationshipCount=10018281 createMillis=13893>


In [ ]:
with driver.session() as session:
    result2 = session.run("CALL gds.degree.stream('myGraph31') YIELD nodeId, score WHERE score <> 0 RETURN gds.util.asNode(nodeId).name AS streamer, score AS chatterIN ORDER BY chatterIN DESC, streamer DESC")
    df = DataFrame(result2.data())
df


,streamer,chatterIN
0,riotgames,194546.0
1,xqcow,171902.0
2,itsbigchase,120726.0
3,esl_csgo,111467.0
4,enardo,109121.0
...,...,...
4533,2crserhatinci,5.0
4534,mehmetaydin90,3.0
4535,elonmusk24577,3.0
4536,farawaykiss,2.0


## Chatter (w/ Stream)

In [52]:
with driver.session() as session:
    result = session.run("CALL gds.graph.create('myGraph32','Stream', {CHATTER: { orientation: 'REVERSE'}})")
    for r in result:
        print(r)

<Record nodeProjection={'Stream': {'properties': {}, 'label': 'Stream'}} relationshipProjection={'CHATTER': {'orientation': 'REVERSE', 'aggregation': 'DEFAULT', 'type': 'CHATTER', 'properties': {}}} graphName='myGraph32' nodeCount=4540 relationshipCount=3014 createMillis=1350>


In [53]:
with driver.session() as session:
    result2 = session.run("CALL gds.degree.stream('myGraph32') YIELD nodeId, score WHERE score <> 0 RETURN gds.util.asNode(nodeId).name AS streamer, score AS chatterIN ORDER BY chatterIN DESC, streamer DESC")
    df = DataFrame(result2.data())
df

,streamer,chatterIN
0,riotgames,87.0
1,xqcow,71.0
2,trainwreckstv,64.0
3,esl_csgo,42.0
4,roshtein,26.0
...,...,...
1298,abby_lu,1.0
1299,8bitdee,1.0
1300,420jenkins,1.0
1301,1pvcs,1.0


# Strongly Connected Components

Alogaritma ini bertujuan untuk mengetahui nomer komponen mana yang paling banyak saling terhubung pada dataset ini

## Moderator

In [ ]:
with driver.session() as session:
    result3md = session.run("CALL gds.alpha.scc.write({nodeProjection: 'Stream', relationshipProjection: 'MODERATOR', writeProperty: 'componentId'}) YIELD setCount, maxSetSize, minSetSize; ")
    df = DataFrame(result3md.data())
df

,setCount,maxSetSize,minSetSize
0,4476,5,1


In [ ]:
with driver.session() as session:
    result4md = session.run("CALL gds.alpha.scc.stream({nodeProjection: 'Stream', relationshipProjection: 'MODERATOR'}) YIELD nodeId, componentId RETURN componentId AS Component ,COUNT(componentId) AS Jumlah_Stream ORDER BY Jumlah_Stream DESC ")
    df = DataFrame(result4md.data())
df

,Component,Jumlah_Stream
0,126,5
1,3757,5
2,211,4
3,1084,4
4,19,3
...,...,...
4471,4535,1
4472,4536,1
4473,4537,1
4474,4538,1


Menampilkan nama yang tercantum pada hasil komponent diatas

In [ ]:
with driver.session() as session:
    result5 = session.run("CALL gds.alpha.scc.stream({ nodeProjection: 'Stream', relationshipProjection: 'MODERATOR'}) YIELD nodeId, componentId WHERE componentId = 126 RETURN gds.util.asNode(nodeId).name AS Name, componentId AS Component ")
    df = DataFrame(result5.data())
df

,Name,Component
0,f1ashko,126
1,romanovalera,126
2,c_a_k_e,126
3,dreadztv,126
4,just_ns,126


In [ ]:
with driver.session() as session:
    result5 = session.run("CALL gds.alpha.scc.stream({ nodeProjection: 'Stream', relationshipProjection: 'MODERATOR'}) YIELD nodeId, componentId WHERE componentId = 3757 RETURN gds.util.asNode(nodeId).name AS Name, componentId AS Component ")
    df = DataFrame(result5.data())
df

,Name,Component
0,mjod,3757
1,drak0ola,3757
2,iinwafqht,3757
3,abuswe7l,3757
4,osamah,3757
